# Extract text from scanned documents with Spark OCR

This notebook will illustrates how to:
* Load example PDF
* Preview it
* Recognize text

## Import OCR transformers and utils

In [0]:
from johnsnowlabs import nlp,medical,visual

08:11:16, INFO Received command c on object id p1


## Define OCR transformers and pipeline
* Transforrm binary data to Image schema using [BinaryToImage](https://nlp.johnsnowlabs.com/docs/en/ocr_pipeline_components#binarytoimage). More details about Image Schema [here](https://nlp.johnsnowlabs.com/docs/en/ocr_structures#image-schema).
* Recognize text using [ImageToText](https://nlp.johnsnowlabs.com/docs/en/ocr_pipeline_components#imagetotext) transformer.

In [0]:
def pipeline():
    
    # Transforrm PDF document to struct image format
    pdf_to_image = visual.PdfToImage()
    pdf_to_image.setInputCol("content")
    pdf_to_image.setOutputCol("image")
    pdf_to_image.setResolution(200)
    pdf_to_image.setPartitionNum(8)

    # Run OCR
    ocr = visual.ImageToText()
    ocr.setInputCol("image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        pdf_to_image,
        ocr
    ])
    
    return pipeline

08:11:18, INFO Received command c on object id p1


## Copy example files from OCR resources to DBFS

In [0]:
import shutil, os
ocr_examples = "/dbfs/FileStore/examples"
#resources = pkg_resources.resource_filename('sparkocr', 'resources')
#if not os.path.exists(ocr_examples):
  #shutil.copytree(resources, ocr_examples)

08:11:22, INFO Received command c on object id p1


In [0]:
%fs ls /FileStore/examples/ocr/pdfs

path,name,size,modificationTime
dbfs:/FileStore/examples/ocr/pdfs/PDftoJSL.pdf,PDftoJSL.pdf,1896330,1695038086000
dbfs:/FileStore/examples/ocr/pdfs/agenda.png,agenda.png,928251,1695124786000
dbfs:/FileStore/examples/ocr/pdfs/alexandria_multi_page.pdf,alexandria_multi_page.pdf,70556,1625483614000
dbfs:/FileStore/examples/ocr/pdfs/fonts.pdf,fonts.pdf,11601,1625483614000
dbfs:/FileStore/examples/ocr/pdfs/genomics/,genomics/,0,1625483613000
dbfs:/FileStore/examples/ocr/pdfs/multiplepages/,multiplepages/,0,1625483614000
dbfs:/FileStore/examples/ocr/pdfs/output.csv,output.csv,0,1695064480000
dbfs:/FileStore/examples/ocr/pdfs/rotated/,rotated/,0,1625483613000
dbfs:/FileStore/examples/ocr/pdfs/tabular-pdf/,tabular-pdf/,0,1625483614000
dbfs:/FileStore/examples/ocr/pdfs/test_document.pdf,test_document.pdf,693743,1625483613000


## Read PDF document as binary file from DBFS

In [0]:
pdf_example = '/FileStore/examples/ocr/pdfs/test_document.pdf'
pdf_example_df = spark.read.format("binaryFile").load(pdf_example).cache()
display(pdf_example_df)

08:11:31, INFO Received command c on object id p1
08:11:32, INFO Received command c on object id p0
08:11:33, INFO Received command c on object id p0
08:11:34, INFO Python Server ready to receive messages
08:11:34, INFO Received command c on object id p0
08:11:34, INFO Received command c on object id p0


path,modificationTime,length,content
dbfs:/FileStore/examples/ocr/pdfs/test_document.pdf,2021-07-05T11:13:33Z,693743,JVBERi0xLjQgCjEgMCBvYmoKPDwKL1BhZ2VzIDIgMCBSCi9UeXBlIC9DYXRhbG9nCj4+CmVuZG9iagoyIDAgb2JqCjw8Ci9UeXBlIC9QYWdlcwovS2lkcyBbIDMgMCBSIDE3IDAgUiBdCi9Db3VudCAyCj4+CmVuZG9iagozIDA= (truncated)


## Preview PDF using _display_images_ function

In [0]:
image_df = visual.PdfToImage().setOutputCol("image").transform(pdf_example_df)
visual.display_images(image_df, limit=3)

08:11:38, INFO Received command c on object id p1
08:11:38, INFO Received command c on object id p0
08:11:39, INFO Received command c on object id p0
08:11:40, INFO Received command c on object id p0
08:11:41, INFO Received command c on object id p0
08:11:42, INFO Received command c on object id p0
08:11:43, INFO Received command c on object id p0
08:11:44, INFO Received command c on object id p0
08:11:45, INFO Received command c on object id p0
08:11:46, INFO Received command c on object id p0
08:11:46, INFO Received command c on object id p0
08:11:47, INFO Received command c on object id p0
08:11:48, INFO Received command c on object id p0
08:11:49, INFO Received command c on object id p0


Origin: dbfs:/FileStore/examples/ocr/pdfs/test_document.pdf 
 Resolution: 300 dpi 
 Width: 5024 px 
 Height: 6615 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/FileStore/examples/ocr/pdfs/test_document.pdf 
 Resolution: 300 dpi 
 Width: 5074 px 
 Height: 6599 px 
 Mode: Greyscale 
 Number of channels: 1

## Run OCR pipelines

In [0]:
from pyspark.ml import PipelineModel

08:12:38, INFO Received command c on object id p1


In [0]:
result = pipeline().transform(pdf_example_df).cache()

08:12:44, INFO Received command c on object id p1


## Display results

In [0]:
display(result.select("pagenum", "text", "confidence"))

08:13:22, INFO Received command c on object id p1
08:13:23, INFO Received command c on object id p0
08:13:24, INFO Received command c on object id p0
08:13:25, INFO Received command c on object id p0
08:13:26, INFO Received command c on object id p0
08:13:27, INFO Received command c on object id p0
08:13:28, INFO Received command c on object id p0
08:13:29, INFO Received command c on object id p0
08:13:30, INFO Received command c on object id p0
08:13:31, INFO Received command c on object id p0


pagenum text confidence 1 Patient Name
Financial Number
Date of Birth
Patient Location

Random Hospital

 

 

 

H & P
Anemia
At baseline
Arthritis

Tylenol for pain. Patient also takes Percocei al home, will add this cn.
Chronic kidney disease
AY baseline. tMonitor while diuresing.
Hypertension
Blood pressures within tolerable ranges.
Pulmonary hypertension
Tricuspid regurgitation
Wild-to-moderaic on echocardiogram last year

Attending physician note-the patient was interviewed and examined. The appropriaic
information in power chart was reviewed. The patient was discussed wilh Dr, Persad.
Patient may have & mild degree of heart failurc. He and his wife were morc concernce with
Ins peripheral edema. He has underlying renal insufficiency as well. We'll try to diurese
him 10 his “dry" weight. We will then try to adjust hie medications to kcep him within a
natrow range of that weight. We will stop his atenolol this point since he ts relatively
bradycardic and observe his heart rate on the cardiac monitor. He will progress with his
care and aclivily as tolerated.

Printed: 7/17/2017 13:01 EDT Page 17 of 42

 

 

 

Vitamin D2 $0,000 intl units (1.25 mg) oral
capsule, 1 TAB, PO, Weexly-Tue

Allergies
shelliisn (cout)

sulfa drug (maculopapular rash)

Social History

 

Ever Smoked Tobacco: Former Smoker
Alcohol use - frequency: None

Drug use: Never

Lab Results

102

3.61H 401 1.23\

07/16/17 05:30 to 07/16/17 05:30

L125

fL 32.4 \

BMP

GLU

NA

K

CL

TOTAL CO2
BUN

CRT

ANION GAP
CA

CBC with diff

WBC
HGB
HCT
RBC
VMICV
MICH
MCHC
RDW
MPV

O7/16/17
05:30

102 mg/dL
143 MMOL/L
3.6 MMOL/L
98 MMOL/L
40 MMOL/L
26 mg/dL.
1.23 mg/dL
5

7.9 mg/dL
07/16/17
05:30

3.4/ nl

10.1 G/DL
32.4 Go

3.41 /PL
95.0 FL
29.6 pg
31.2 %

15.9 %

10.7 FL

PowerChart
 79.89152243477957 0 Patient Nam
Financial Numbe

Random Hospital Date of Birth

Patient Location

 
 
 
 

 

 

 

Chief Complaint
Shortness of breath

History of Present Illness

 

Patient is an 84-year-old male wilh a past medical history of hypertension, HFpEF las
Known EF 55%c, mild to moderate TA, pulmonary hypertension, permanent atrial
fibrillation on Eliquis, history of GI blesd, CK-I48, and anemia who presents with full weeks
oi ccneralized fatigue and fcoling unwell. He also notes some shortness oi breath and
worsening dyspnea wilh minimal exertion. His major complaints are shoulder ard joint
pains. diffuscly. He also complains of "bone pain’. He denics having any fevers or chills.

e demes having any chest pain, palpitations. He denies any worse extremity
swelling than his baseline. He states he’s bccn compliant with his medications. Although
he stales he ran out of his Eliquis & few weeks ago, He denies having any blood in his
stools or mc!cna, although he does take iron pills and states his stools arc irequently black.
His hemeglobin is al baseline.

Twelve-lead EKG showing atrial fibrillation, RBBB, LAFB, PVC. Chest x-ray showing new
small right creater than left pleural effusions with mild pulmonary vascular congestion. BNP
increased to 2800, up fram 1900. Tropoain 0.03. Renal function at baseline. Hemoglobin
ai baseline.

She normally takes 80 mg of oral Lasix daily. He was given 80 mg of IV Lasix in the ED.
He is currently net nogative close to 1 L. He is stillon 2 L nasal cannula.

y s 5
A 10 system review of sysiems was completed and negative except as documented in HPI.

Physical Exam

 

Vilals & Measurements
T: 36.8 °C (Oral) TMIN: 36.8 "C (Oral) TMAX: 37.0 °C (Oral) HR: 54 RR: 17
BP: 140/63 WT: 100.3 KG
Pulse Ox: 100 % Oxygen: 2 Limin via Nasal Cannula
GENERAL: no acule distress
HEAD: normecephalic
EYES‘EARS/NOSE/THAROAT: oupils are equal. normal oropharynx
NECK: normal inspection
RESPIRATORY: no respiratory distress, no rales on my exam
CARDIOVASCULAR: irregular. brady. no murmurs. rubs or galleps
ABDOMEN: soft, non-tendes
EXTREMITIES: Bilateral chronic venous stasis changes
NEUROLOGIC: alert and osieniec x 3, no gross motar or sensory deficils

 

AssessmenvP

## Clear cache

In [0]:
result.unpersist()
pdf_example_df.unpersist()

08:13:53, INFO Received command c on object id p1


DataFrame[path: string, modificationTime: timestamp, length: bigint, content: binary]